# Caffe学习系列：命令行解析

Caffe的运行提供三种接口：C++接口（命令行）、Python接口和Matlab接口。本文先对命令行进行解析，后续会依次介绍其他两个接口。

Caffe的C++主程序（caffe.cpp）放在根目录下的tools文件夹内，当然还有一些其他的功能文件，如：convert_imageset.cpp，train_net.cpp，test_net.cpp等也放在这个文件夹内。经过编译后，这些文件都被编译成了可执行文件，放在了./build/tools/文件夹内。因此我们要执行caffe程序，都需要加./build/tools/前缀。

如：
```
sudo sh ./build/tools/caffe train --solver=examples/mnist/train_lenet.sh
```

Caffe程序的命令行执行格式如下：
```
caffe <command> <args>
```
其中的&lt;command&gt;有这样四种：
- train
- test
- device-query
- time

对应的功能为：
- train——训练或finetune模型（model）。
- test——测试模型。
- device_query——显示GPU信息。
- time——显示程序执行时间。

其中的&lt;args&gt;参数有：
- -solver
- -gpu
- -snapshot
- -weights
- -iterations
- -model
- -sighup_effect
- -sigint_effect

注意前面有个-符号。对应的功能为：

-solver：必须参数。一个Protocol Buffer类型的文件，即模型的配置文件。如：
```
./build/tools/caffe train --solver examples/mnist/lenet_solver.prototxt
```

-gpu：可选参数。该参数用来指定用哪一块GPU运行，根据GPU的id进行选择，如果设置为'-gpu all'则使用所有的GPU运行。如使用第二块GPU运行：
```
./build/tools/caffe train --solver examples/mnist/lenet_solver.prototxt --gpu 2
```

-snapshot：可选参数。该参数用来从快照（snapshot）中恢复训练。可以在solver配置文件设置快照，保存solverstate。如：
```
./build/tools/caffe train --solver examples/mnist/lenet_solver.prototxt --snapshot examples/mnist/lenet_iter_5000.solverstate
```

-weights：可选参数。用预先训练好的权重来fine-tuning模型，需要一个caffemodel，不能和-snapshot同时使用。如：
```
./build/tools/caffe train --solver examples/finetuning_on_flickr_style/solver.prototxt --weights models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel
```

-iterations：可选参数，迭代次数，默认为50。如果在配置文件中没有设定迭代次数，则默认迭代50次。

-model：可选参数，定义在Protocol Buffer文件中的模型。也可以在solver配置文件中指定。

-sighup_effect：可选参数。用来设定当程序发生挂起事件时执行的操作，可以设置为snapshot，stop和none，默认为snapshot。

-sigint_effect：可选参数。用来设定当程序发生键盘中止事件时（ctrl+c）执行的操作，可以设置为snapshot，stop或none，默认为stop。

刚才举例了一些**train**参数的例子，现在我们来看看其他三个&lt;command&gt;：

**test**参数用在测试阶段，用于最终结果的输出，在模型配置文件中我们可以设定需要输出accuracy还是loss。假设我们要在验证集中验证已经训练好的模型，就可以这样写：
```
./build/tools/caffe test --model examples/mnist/lenet_train_test.prototxt --weights examples/mnist/lenet_iter_10000.caffemodel --gpu 0 --iterations 100
```
这个例子比较长，不仅用到了test参数，还用到了--model，--weights，--gpu和--iterations四个参数。意思是利用训练好了的权重（--weights），输入到测试模型中（--model），用编号为0的gpu（--gpu）测试100次（--iterations）。

**time**参数用来在屏幕上显示程序运行时间。如：
```
./build/tools/caffe time --model examples/mnist/lenet_train_test.prototxt --iterations 10
```
这个例子用来在屏幕上显示LeNet模型迭代10次所使用的时间。包括每次迭代的forward和backward所用的时间，也包括每层forward和backward所用的平均时间。

```
./build/tools/caffe time --model examples/mnist/lenet_train_test.prototxt -gpu 0
```
这个例子用来在屏幕上显示LeNet模型用GPU迭代50次所使用的时间。

```
./build/tools/caffe time --model examples/mnist/lenet_train_test.prototxt --weights examples/mnist/lenet_iter_10000.caffemodel --gpu 0 --iterations 10
```
利用给定的权重，利用第一块GPU，迭代10次LeNet模型所用的时间。

**device_query**参数用来诊断GPU信息：
```
./build/tools/caffe device_query --gpu 0
```

最后我们来看两个关于GPU的例子：
```
./build/tools/caffe train --solver examples/mnist/lenet_solver.prototxt --gpu 0,1
```
```
./build/tools/caffe train --solver examples/mnist/lenet_solver.prototxt --gpu all
```
这两个例子表示：用两块或多块GPU来平行运算，这样速度会快很多。但是如果你只有一块或没有GPU，就不要加--gpu参数了，加了反而慢。

最后，在Linux下，本身就有一个time命令，因此可以结合进来使用，因此我们运行mnist例子的最终命令是（一块gpu）：
```
sudo time ./build/tools/caffe train --solver examples/mnist/lenet_solver.prototxt
```